In [ ]:
import tensorflow as tf
from tensorflow.keras import backend, optimizers
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
import os

In [ ]:
img_dir = '../data'
train_dir = os.path.join(img_dir, 'Training')
test_dir = os.path.join(img_dir, 'Test')

BATCH_SIZE = 32
IMG_SIZE = (299, 299)

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  shuffle=True,
  batch_size=BATCH_SIZE,
  image_size=IMG_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  shuffle=True,
  batch_size=BATCH_SIZE,
  image_size=IMG_SIZE
)

In [ ]:
train_dataset = train_dataset.map(lambda x, y: (x / 255.0, y))
validation_dataset = validation_dataset.map(lambda x, y: (x / 255.0, y))

In [ ]:
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
backend.clear_session()

conv_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [ ]:
InceptionV3_model_input = conv_base.output
pooling = GlobalAveragePooling2D()(InceptionV3_model_input)
dense = Dense(512, activation='relu')(pooling)
output = Dense(131, activation='softmax')(dense)

In [ ]:
model_InceptionV3 = Model(inputs=conv_base.input, outputs=output)
model_InceptionV3.summary()

In [ ]:
model_InceptionV3.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizers.SGD(learning_rate=1e-4, momentum=0.9),
    metrics=['accuracy']
)

In [ ]:
history = model_InceptionV3.fit(
    train_dataset,
    epochs=5,
    validation_data=validation_dataset,
    callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)]
)

In [ ]:
model_InceptionV3.save('model_name')